In [53]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
from time import sleep
from random import randint
import numpy as np
import re

In [54]:
EventName=[]
Date = []
CityCountry= []
Exhibits= []
URL = []
pages = list(range(1,21))
for page in pages:
  req = requests.get("https://www.jetro.go.jp/en/database/j-messe/country/asia/?dnumber=&sort=&dnumber=&sort=&_page={}".format(page)).text    
  soup = BeautifulSoup(req,'html.parser')

  titles = soup.find_all('p',{'class':'font18 font_bold'})
  for title in titles:
    eventname= title.text
    EventName.append(eventname)

  commonclass = soup.find_all('dl',class_='w80')
  for dlitem in commonclass: 
    date= dlitem.find("dt",text="Date").findNext("dd").string
    Date.append(date)
  for dlitem in commonclass: 
    city= dlitem.find("dt",text="City/country").findNext("dd").string
    CityCountry.append(city)
  for dlitem in commonclass: 
    exhibits= dlitem.find("dt",text="Exhibits").findNext("dd").string
    Exhibits.append(exhibits)

  links =  soup.findAll('a', attrs={'href': re.compile("^/en/database/j-messe/tradefair/detail/")})
  for link in links:
    f_link = link.get('href')
    if f_link:
      url= 'https://www.jetro.go.jp' + f_link
      print(url)
    URL.append(url)

https://www.jetro.go.jp/en/database/j-messe/tradefair/detail/106814
https://www.jetro.go.jp/en/database/j-messe/tradefair/detail/119662
https://www.jetro.go.jp/en/database/j-messe/tradefair/detail/119661
https://www.jetro.go.jp/en/database/j-messe/tradefair/detail/119658
https://www.jetro.go.jp/en/database/j-messe/tradefair/detail/119657
https://www.jetro.go.jp/en/database/j-messe/tradefair/detail/119655
https://www.jetro.go.jp/en/database/j-messe/tradefair/detail/119218
https://www.jetro.go.jp/en/database/j-messe/tradefair/detail/108634
https://www.jetro.go.jp/en/database/j-messe/tradefair/detail/108631
https://www.jetro.go.jp/en/database/j-messe/tradefair/detail/108127
https://www.jetro.go.jp/en/database/j-messe/tradefair/detail/119642
https://www.jetro.go.jp/en/database/j-messe/tradefair/detail/119641
https://www.jetro.go.jp/en/database/j-messe/tradefair/detail/119636
https://www.jetro.go.jp/en/database/j-messe/tradefair/detail/119635
https://www.jetro.go.jp/en/database/j-messe/trad

In [55]:
print(len(EventName))
print(len(Date))
print(len(CityCountry))
print(len(Exhibits))
print(len(URL))

400
400
400
400
400


In [113]:
df = {'EventName':EventName[slice(300)],'Date':Date[slice(300)],'City/Country':CityCountry[slice(300)],'Exhibits':Exhibits[slice(300)],'URL':URL[slice(300)]}
event_df = pd.DataFrame(data = df)

In [114]:
event_df

,EventName,Date,City/Country,Exhibits,URL
0,\n HOFEX 2021 - Asi...,"May 12, 2021 - May 14, 2021",\n ...,"Food & Drinks, Wines/Spirits/Beer, Coffee & Te...",https://www.jetro.go.jp/en/database/j-messe/tr...
1,\n Care Show Japan2...,"February 8, 2022 - February 10, 2022",\n ...,"Nursing food, swallowing food, nursing system,...",https://www.jetro.go.jp/en/database/j-messe/tr...
2,\n CIOE 2021 - Chin...,"September 1, 2021 - September 3, 2021",\n ...,"Optical Communications, Precision Optics, Lase...",https://www.jetro.go.jp/en/database/j-messe/tr...
3,\n Japan Jewellery ...,"September 1, 2021 - September 3, 2021",\n ...,"Fine Jewellery, Diamonds & Diamond Jewellery, ...",https://www.jetro.go.jp/en/database/j-messe/tr...
4,\n nano tech 2022\n...,"January 26, 2022 - January 28, 2022",\n ...,"All about nano-scale technologies: Materials, ...",https://www.jetro.go.jp/en/database/j-messe/tr...
...,...,...,...,...,...
295,\n Small and Medium...,"June 30, 2021 - July 2, 2021",\n ...,"metal processing, resin processing and chemica...",https://www.jetro.go.jp/en/database/j-messe/tr...
296,\n Maintenance & Re...,"September 22, 2021 - September 24, 2021",\n ...,"Products, technologies, and services in the sc...",https://www.jetro.go.jp/en/database/j-messe/tr...
297,\n 8th TOKYO FASHIO...,"March 23, 2021 - March 25, 2021",\n ...,"Women's Wear, Inner cloth, Lingerie, Stole, Sc...",https://www.jetro.go.jp/en/database/j-messe/tr...
298,\n JAPAN INTERNATIO...,"March 9, 2021 - March 12, 2021",\n ...,Franchisers (l.Food 2.Retail 3.Service) seekin...,https://www.jetro.go.jp/en/database/j-messe/tr...


In [115]:
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\a]", " ")
    for idx, line in enumerate(df[text_field]):
      df[text_field][idx] = ' '.join(line.split())
    return df

In [116]:
standardize_text(event_df, "EventName")
standardize_text(event_df, "City/Country")

,EventName,Date,City/Country,Exhibits,URL
0,HOFEX 2021 Asia's Leading Food Hospitality Tra...,"May 12, 2021 - May 14, 2021",Hong Kong China Asia,"Food & Drinks, Wines/Spirits/Beer, Coffee & Te...",https://www.jetro.go.jp/en/database/j-messe/tr...
1,Care Show Japan2022,"February 8, 2022 - February 10, 2022",Tokyo Japan Asia,"Nursing food, swallowing food, nursing system,...",https://www.jetro.go.jp/en/database/j-messe/tr...
2,CIOE 2021 China International Optoelectronic E...,"September 1, 2021 - September 3, 2021",Shenzhen China Asia,"Optical Communications, Precision Optics, Lase...",https://www.jetro.go.jp/en/database/j-messe/tr...
3,Japan Jewellery Fair 2021,"September 1, 2021 - September 3, 2021",Kofu Japan Asia,"Fine Jewellery, Diamonds & Diamond Jewellery, ...",https://www.jetro.go.jp/en/database/j-messe/tr...
4,nano tech 2022,"January 26, 2022 - January 28, 2022",Tokyo Japan Asia,"All about nano-scale technologies: Materials, ...",https://www.jetro.go.jp/en/database/j-messe/tr...
...,...,...,...,...,...
295,Small and Medium Enterprise techno fair in Kyu...,"June 30, 2021 - July 2, 2021",Kitakyushu (Fukuoka) Japan Asia,"metal processing, resin processing and chemica...",https://www.jetro.go.jp/en/database/j-messe/tr...
296,Maintenance Resilience ASIA 2021,"September 22, 2021 - September 24, 2021",Bangkok Thailand Asia,"Products, technologies, and services in the sc...",https://www.jetro.go.jp/en/database/j-messe/tr...
297,8th TOKYO FASHION WEAR EXPO MARCH,"March 23, 2021 - March 25, 2021",Tokyo Japan Asia,"Women's Wear, Inner cloth, Lingerie, Stole, Sc...",https://www.jetro.go.jp/en/database/j-messe/tr...
298,JAPAN INTERNATIONAL FRANCHISE SHOW 2021,"March 9, 2021 - March 12, 2021",Tokyo Japan Asia,Franchisers (l.Food 2.Retail 3.Service) seekin...,https://www.jetro.go.jp/en/database/j-messe/tr...


In [117]:
event_df.to_csv('/content/Events.csv')